In [1]:
import os
from pathlib import Path
import re
from itertools import combinations
from functools import lru_cache
from collections import defaultdict, deque

FOLDER = Path(os.path.dirname(os.path.realpath("__file__"))) / 'data'
in_file = 'day16.txt'

## Part One

In [2]:
graph = {}

with open(FOLDER/in_file) as f:
    for line in f:
        key, *tunnels = re.findall(r'[A-Z]{2}', line)
        rate = int(re.search(r'\d+', line).group())
        graph[key] = {'rate': rate, 'tunnels':set(tunnels)}

def bfs(graph, S, E):
    '''
    Breadth-first search of graph
    graph should be a dictionary-based adjacency list.
    S and E are start and end represented as (x, y) tuples.
    '''
    queue = deque([(S, 0, [S])])
    marked = set()
    while queue:
        point, count, path = queue.popleft()
        if point == E:
            return count, path
        for p in graph[point]['tunnels']:
            if p not in marked:
                queue.append([p, count+1, path + [p]])
                marked.add(p)
                
class Cave:
    def __init__(self, graph):
        self.graph = graph        
        self.distance_cache = defaultdict(dict)

        # the locations with a valve that could be turned on
        self.targets = frozenset([k for k, v in self.graph.items() if v['rate'] > 0])
       
    def distance_between(self, start, end):
        distance = self.distance_cache[start].get(end)
        if distance is None:
            distance, path = bfs(self.graph, start, end)
            self.distance_cache[start][end] = distance
        return distance
        

In [3]:
cave = Cave(graph)
time = 30
initial_targets = cave.targets

@lru_cache(maxsize=None)
def max_tour(pos, time, targets):
    if not targets:
        return 0
    if time <=0:
        return 0
    
    max_flow = 0
    
    for t in targets:
        rate = cave.graph[t]['rate']
        distance = cave.distance_between(pos, t)
        
        if (1 + distance) > time:
            continue
        
        total_flow = rate * (time - distance - 1)
        total_flow += max_tour(t, time-distance-1, targets - {t})
        
        if total_flow > max_flow:
            max_flow = total_flow
        
    return max_flow

max_tour('AA', time, initial_targets)

1737

# Part 2

Use the same route finding and brute force through all partitions of the open valves.
Made (barely) tractable by LRU Cache.

In [4]:
s = cave.targets
time = 26

max_flow = 0

for combs in (combinations(s, r) for r in range(len(s) + 1)):
    for comb in combs:
        diff = s.difference(comb)
        total = max_tour('AA', time, diff) + max_tour('AA', time, frozenset(comb))
        if total > max_flow:
            max_flow = total

max_flow    

2216